<a href="https://colab.research.google.com/github/tlsgptj/CJONs/blob/chatbot/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers[sentencepiece]

In [ ]:
!pip install sentencepiece transformers[sentencepiece]

In [ ]:
import transformers
import torch
import sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# 챗봇에 사용할 BERT 모델과 tokenizer를 로드합니다.
chatbot_model_name = "bert-base-multilingual-cased"  # 다국어 BERT 모델
chatbot_tokenizer = AutoTokenizer.from_pretrained(chatbot_model_name)
chatbot_model = AutoModelForSequenceClassification.from_pretrained(chatbot_model_name)

In [ ]:
from transformers import AutoTokenizer

# Replace 'your_model_name' with the name of the model you want to use
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

# Now you should be able to use the tokenizer without errors


In [ ]:
pip list | grep sentencepiece

In [ ]:
def chatbot_response(input_text):
    inputs = chatbot_tokenizer(input_text, return_tensors="pt")
    outputs = chatbot_model(**inputs)
    response = chatbot_tokenizer.decode(outputs.logits.argmax(dim=-1)[0])
    return response

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset

# 데이터 로딩
data = [("What is wine?", "Wine is an alcoholic beverage made from fermented grapes or other fruits.", 0),
("What are the different types of wine?", "There are several types of wine, including red, white, rosé, sparkling, and dessert wines.", 1),
("How is wine made?", "Wine is made through the fermentation of grape juice, where yeast converts sugar into alcohol.", 2),
("What is the difference between red and white wine?", "Red wine is made from red or black grapes and is fermented with the grape skins, while white wine is made from green or yellow grapes and is fermented without the skins.", 3),
("How should I store wine at home?", "Wine should be stored in a cool, dark place, preferably on its side to keep the cork moist.", 4),
("What are tannins in wine?", "Tannins are natural compounds found in grape skins, seeds, and stems, which give wine its astringency and bitterness.", 5),
("What is the ideal serving temperature for red wine?", "Red wine is typically served at room temperature, around 60-65°F (15-18°C).", 6),
("How long can I keep an opened bottle of wine?", "Once opened, wine can be kept for a few days to a week, depending on the type. Red wines generally last longer than white wines.", 7),
("What is the best wine to pair with steak?", "A bold red wine, such as Cabernet Sauvignon or Malbec, pairs well with steak.", 8),
("What is the difference between dry and sweet wine?", "Dry wine has very little residual sugar, while sweet wine has higher sugar content.", 9),
("What is the difference between Cabernet Sauvignon and Merlot?", "Cabernet Sauvignon is a full-bodied red wine with bold tannins, while Merlot is softer and more approachable with a medium body.", 10),
("What is the best wine to pair with cheese?", "Red wines like Cabernet Sauvignon, Merlot, or Syrah/Shiraz pair well with hard cheeses, while white wines like Chardonnay or Sauvignon Blanc go well with soft cheeses.", 11),
("Can I age sparkling wine?", "Most sparkling wines are meant to be consumed young and fresh, and aging may not improve their taste.", 12),
("What is the difference between a wine's aroma and its nose?", "The aroma refers to the overall smell of the wine, while the nose specifically refers to the smell of the wine in the glass.", 13),
("Can I use wine as a marinade?", "Yes, wine can be used as a marinade to add flavor and tenderize meat.", 14),
("What is the best wine to pair with roast chicken?", "Chardonnay or Pinot Noir pair well with roast chicken.", 15),
("How do I clean wine glasses?", "To clean wine glasses, hand wash them with warm water and mild detergent, and let them air dry to avoid leaving streaks or residue.", 16),
("What is the best wine to pair with sushi?", "A dry Riesling or a sparkling wine like Champagne go well with sushi.", 17),
("What is the difference between a wine's aroma and its taste?", "The aroma refers to the smell of the wine, while the taste includes the flavor and mouthfeel experienced on the palate.", 18),
("Can I age white wine?", "Some high-quality white wines can be aged, but most white wines are meant to be consumed within a few years of purchase.", 19),
("Please describe the taste of wine.", "Wine has rich fruit flavors and smooth tannins.", 20),
("How does wine's acidity feel?", "Wine's acidity is tart and refreshing.", 20),
("What is tannin?", "Tannin is a component in wine that gives it a bitter taste, often found in grape skins or seeds.", 20),
("How is the sweetness of wine determined?", "The sweetness of wine is determined by the amount of residual sugar it contains.", 20),
("Which wine pairs well with meat?", "Red wine pairs well with meat.", 20),
("What does 'body' mean in wine?", "The body of wine refers to its weight or texture, ranging from light-bodied to full-bodied.", 20),
("Why does wine taste bitter?", "Wine can taste bitter due to the presence of tannins.", 20),
("Does higher alcohol content in wine affect its taste?", "Higher alcohol content in wine can make it warmer and more intense in flavor.", 20),
("Does wine taste better with age?", "Some wines can improve with age, gaining more complexity and subtle flavors.", 20),
("Is it okay to drink wine warm?", "Wine is generally best served slightly chilled.", 20),
("Which is better, white wine or red wine?", "It depends on individual preferences, but red wines are usually bolder and more complex.", 20),
("How do you decant wine?", "Decanting involves transferring wine to a separate container, like a decanter or a glass pitcher.", 20),
("Why is decanting wine beneficial?", "Decanting can soften the wine's acidity and tannins, enhancing its flavors.", 20),
("Does the order of wine tasting matter?", "The order of wine tasting is important to fully appreciate its characteristics.", 20),
("At what temperature should wine be served?", "White wine should be served slightly chilled, while red wine can be served at slightly warmer temperatures.", 20),
("How should you taste wine to fully experience its flavor?", "Before sipping, try to smell the wine and then take a small sip to savor the taste.", 20),
("Can you suggest wine and food pairings?", "Wines with strong fruit flavors go well with seafood, while red wine pairs nicely with steak.", 20),
("How long can you store wine?", "The storage time varies depending on the type of wine, but generally, 2-10 years of aging is common.", 20),
("Why is aging wine in oak barrels preferred?", "Aging wine in oak barrels adds flavors like vanilla and enhances its smoothness.", 20),
("How is sweetness created in wine?", "Sweetness in wine is created when there is residual sugar in the wine after fermentat", 20)]  # 라벨링한 데이터를 리스트 형태로 저장

# BERT 토크나이저
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 입력 문장과 라벨을 토치 데이터셋으로 변환
class WineDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data[idx][0]
        label = self.data[idx][2]

        inputs = tokenizer(input_text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        label = torch.tensor(int(label))

        return inputs, label

# 데이터셋 준비
wine_dataset = WineDataset(data)
train_loader = DataLoader(wine_dataset, batch_size=8, shuffle=True)

NUM_LABELS = 21
NUM_EPOCHS = 21

# BERT 모델 불러오기
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=NUM_LABELS)

# Optimizer 설정 및 학습
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

inputs = {k: v.to(device) for k, v in inputs.items()}
labels = labels.to(device)


def collate_fn(batch):
    inputs, labels = zip(*batch)  # tuple을 풀어줍니다.
    inputs = {k: torch.cat([item[k] for item in inputs], dim=0) for k in inputs[0]}  # batch 단위로 concatenate
    labels = torch.stack(labels, dim=0)  # batch 단위로 stack

    return inputs, labels

train_loader = DataLoader(wine_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

for epoch in range(NUM_EPOCHS):
    model.train()
    for inputs, labels in train_loader:
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# 학습된 모델을 이용하여 챗봇 구현
conversation=[]
def wine_chatbot(question):
    model.eval()
    inputs = tokenizer(question, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits).item()

    if predicted_label < 0 or predicted_label >= NUM_LABELS:
        return "I apologize, but I couldn't find an answer to that question."

    # 대화 기록 저장
    conversation.append((question, data[predicted_label][1]))

    return data[predicted_label][1]

# 대화 기록을 출력하는 함수
def print_conversation():
    print("대화 기록:")
    for i, (user_input, bot_response) in enumerate(conversation):
        print(f"{i + 1}. 사용자: {user_input}")
        print(f"   챗봇: {bot_response}")
        print()

# 챗봇 테스트
question = "What is the difference between red and white wine?"
predicted_label = wine_chatbot(question)
print("Predicted label:", predicted_label)

In [ ]:
torch.save(model.state_dict(), 'wine_chatbot_model.pt')

In [ ]:
from transformers import BertTokenizer, BertModel

# 와인 BERT 모델 로드
wine_bert_model_name = "path_to_pretrained_wine_bert"  # 와인 BERT 모델의 경로나 이름
wine_tokenizer = BertTokenizer.from_pretrained(wine_bert_model_name)
wine_model = BertModel.from_pretrained(wine_bert_model_name)

# 챗봇 BERT 모델 로드
chatbot_bert_model_name = "path_to_pretrained_chatbot_bert"  # 챗봇 BERT 모델의 경로나 이름
chatbot_tokenizer = BertTokenizer.from_pretrained(chatbot_bert_model_name)
chatbot_model = BertModel.from_pretrained(chatbot_bert_model_name)

# 챗봇 BERT에 와인 BERT 삽입
# 예를 들어, 챗봇 BERT의 첫 번째 레이어에 와인 BERT를 삽입하겠습니다.
chatbot_model.bert.embeddings.word_embeddings.weight = wine_model.bert.embeddings.word_embeddings.weight

# 이제 챗봇 BERT를 사용하여 대화를 수행할 수 있습니다.
input_text = "와인 추천해줘"
inputs = chatbot_tokenizer(input_text, return_tensors="pt")
outputs = chatbot_model(**inputs)
# 결과로 나온 outputs를 해석하여 적절한 답변을 생성할 수 있습니다.

# 이후 모델을 훈련하고 세부적인 파라미터 조정 등을 진행해야 합니다.

In [ ]:
!pip install transformers

In [ ]:
#내가 해야 할 일
#1. 받은 와인 테스트 데이터를 통해
#2. test.csv를 통해 챗봇 구현하기

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, BertConfig
import numpy as np
import pandas as pd

# 와인 맛 BERT 모델 로드 함수
def load_wine_model(model_path):
    config = BertConfig.from_json_file(f"{model_path}.json")
    model = tf.keras.models.load_model(f"{model_path}.h5")
    return model, config

# 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained("kykim/bert-kor-base")

# 와인 데이터 읽어오기
def load_wine_data(file_path):
    df = pd.read_csv(file_path)
    return df

# 와인 추천 함수
def recommend_wines(taste_scores, wine_data):
    recommended_wines = []
    for idx, score in enumerate(taste_scores):
        if score >= 0.5:  # 임계값을 조정하여 맛에 따른 추천 여부 결정
            recommended_wines.extend(wine_data[wine_data['맛'] == taste_categories[idx]]['와인 이름'])
    return recommended_wines

# 사용자 입력에 대한 응답 생성 함수
def generate_response(user_input, wine_data, wine_models):
    # 사용자 입력을 BERT 모델 입력 형식에 맞게 전처리
    inputs = tokenizer(user_input, return_tensors='tf', padding=True, truncation=True)

    # 모든 와인 맛 BERT 모델을 통해 예측
    taste_scores = []
    for model, _ in wine_models:
        outputs = model(inputs['input_ids'])
        logits = outputs.logits.numpy()[0]
        taste_scores.append(logits)

    # 추천할 와인 종류 추출
    recommended_wines_list = recommend_wines(np.array(taste_scores).mean(axis=0), wine_data)

    if recommended_wines_list:
        response = "다음과 같은 와인들이 어울릴 것 같아요:\n" + "\n".join(recommended_wines_list)
    else:
        response = "죄송하지만, 원하시는 와인을 찾을 수 없습니다."

    return response

# 챗봇 동작
def chatbot():
    print("안녕하세요! 와인 추천 챗봇입니다. 어떤 종류의 와인을 추천 받고 싶으신가요?")

    # 와인 데이터 읽어오기
    wine_data = load_wine_data("/content/test.csv")

    # 와인 맛 BERT 모델 로드
    wine_models = []
    for model_name in ["model_sweet", "model_bitter", "model_sour"]:
        model, _ = load_wine_model(f"/path/to/{model_name}")
        wine_models.append((model, _))

    while True:
        user_input = input("질문: ")

        if user_input.lower() == '종료':
            print("와인 추천 챗봇을 종료합니다.")
            break

        response = generate_response(user_input, wine_data, wine_models)
        print("답변:", response)

# 챗봇 실행
if __name__ == "__main__":
    chatbot()


In [ ]:
pip install transformers

In [ ]:
#이거 찐임
import csv

# 와인 데이터 읽어오기
def load_wine_data(file_path):
    wine_data = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            wine_name = row['WineName']
            bitterness = float(row['Tannins_all'])
            sweetness = float(row['Sweet_all'])
            sourness = float(row['Acidic_all'])
            lightness = float(row['Light_all'])
            ex_bitterness = float(row['Tannins_ex'])
            ex_sweetness = float(row['Sweet_ex'])
            ex_sourness = float(row['Acidic_ex'])
            ex_lightness = float(row['Light_ex'])
            bg_lightness = float(row['Light_bg'])
            bg_sourness = float(row['Acidic_bg'])
            bg_sweetness = float(row['Sweet_bg'])
            bg_bitterness = float(row['Tannins_bg'])
            key_sweetness = float(row['Sweet_key'])
            key_bitterness = float(row['Tannins_key'])
            key_sourness = float(row['Acidic_key'])

            wine_data[wine_name] = {
                '쓴맛': bitterness,
                '단맛': sweetness,
                '신맛': sourness,
                '바디감' : lightness,
                '전문가 단맛' : ex_sweetness,
                '전문가 쓴맛' : ex_bitterness,
                '전문가 신맛' : ex_sourness,
                '전문가 바디감' : ex_lightness,
                '초심자 단맛' : bg_sweetness,
                '초심자 쓴맛' : bg_bitterness,
                '초심자 신맛' : bg_sourness,
                '초심자 바디감' : bg_lightness,
                '키워드 단맛' : key_sweetness,
                '키워드 신맛' : key_sourness,
                '키워드 쓴맛' : key_bitterness
            }
    return wine_data

def recommend_wine(user_preference, wine_data, threshold=10):
    recommended_wines = []
    for wine_name, wine_info in wine_data.items():
        user_sweetness = wine_info['단맛']
        user_bitterness = wine_info['쓴맛']
        user_sourness = wine_info['신맛']
        user_lightness = wine_info['바디감']

    if (
          abs(user_preference['단맛']) <= threshold and
          abs(user_preference['쓴맛']) <= threshold and
          abs(user_preference['신맛']) <= threshold and
          abs(user_preference['바디감']) <= threshold
        ):

            recommended_wines.append(wine_name)

    return recommended_wines

def chatbot():
    file_path = '/content/WineDataset.csv'
    wine_data = load_wine_data(file_path)

    print("안녕하세용 전 와댕이에용 멍멍!! 종료하려면 '종료'라고 입력하세요!")

    while True:
        print("와인의 맛을 알려주세요! 멍멍!!")
        user_sourness = int(input("단맛은요? (0 ~ 100 사이의 값을 입력하세요): ")) / 100
        user_bitterness = int(input("쓴맛은요? (0 ~ 100 사이의 값을 입력하세요): ")) / 100
        user_sweetness = int(input("신맛은요? (0 ~ 100 사이의 값을 입력하세요): ")) / 100
        user_lightness = int(input("바디감은요? (0 ~ 100 사이의 값을 입력하세요): ")) / 100

        threshold = 10

        recommended_wines = recommend_wine({'단맛': user_sweetness, '쓴맛': user_bitterness, '신맛': user_sourness, '바디감' : user_lightness}, wine_data, threshold)

        if recommended_wines:
            print("와인 추천 결과에요! 마음에 들었으면 좋겠네용:")
            for wine in recommended_wines:
              wine_info = wine_data[wine]
              print(f"이 와인을 추천해요! \n와인 이름: {wine}")
              print(f"\n이 와인의 전체적인 맛은 다음과 같아요 \n단맛 : {wine_info['단맛']}, 쓴맛 : {wine_info['쓴맛']}, 신맛 : {wine_info['신맛']}, 바디감 : {wine_info['바디감']}이고 \n\n전문가들은 이렇게 평가했어요! \n단맛 : {wine_info['전문가 단맛']}, 쓴맛 : {wine_info['전문가 쓴맛']}, 신맛 : {wine_info['전문가 신맛']} \n")
              print("초심자들은 이렇게 평가했어요!")
              print(f"단맛 : {wine_info['초심자 단맛']}, 쓴맛 : {wine_info['초심자 쓴맛']}, 신맛 : {wine_info['초심자 신맛']} \n\n이거에요! 멍!")

        else:
            print("해당하는 와인 정보를 찾을 수 없어요. ㅜㅜ")

        user_input = input("또 궁금한거 있으세요? (예/아니오): ")
        if user_input.lower() == '아니오':
            print("와댕이 이제 가볼께용! 멍!")
            break

if __name__ == "__main__":
    chatbot()


In [ ]:
9#모델 저장
!pip install joblib

In [ ]:
import joblib

# 챗봇 모델 저장
def save_chatbot_model(model, file_path):
    try:
        joblib.dump(model, file_path)
        print("챗봇 모델이 성공적으로 저장되었습니다.")
    except Exception as e:
        print(f"챗봇 모델 저장 중 오류 발생: {e}")

# 챗봇 모델 로드
def load_chatbot_model(file_path):
    try:
        model = joblib.load(file_path)
        print("챗봇 모델이 성공적으로 로드되었습니다.")
        return model
    except Exception as e:
        print(f"챗봇 모델 로드 중 오류 발생: {e}")
        return None


In [ ]:
# 챗봇 모델 저장
chatbot_model = chatbot  # chatbot 함수를 저장하려면 chatbot_model = chatbot 대신 사용
save_chatbot_model(chatbot_model, 'wadeang_model.joblib')

# 챗봇 모델 로드
loaded_chatbot_model = load_chatbot_model('chatbot_model.joblib')

# 챗봇 모델 사용
if loaded_chatbot_model:
    loaded_chatbot_model()
